In [2]:
import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
%matplotlib inline

In [3]:
root_path="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/database"

In [4]:
def mfcc_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features

In [5]:
#NOT USED
def fft_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    fft = np.fft.fft(signal)
    
    return fft

In [6]:
#NOT USED
signal, sample_rate = librosa.load('database/ac/ac_AHCG_FSRPNCW.wav')
ipd.Audio('database/ac/ac_AHCG_FSRPNCW.wav')
fft = np.fft.fft(signal)
fft

array([ 3.96959355+0.j        , -1.57368256-0.43014124j,
       -0.5562213 +0.51962332j, ...,  0.00995858+0.12657352j,
       -0.5562213 -0.51962332j, -1.57368256+0.43014124j])

In [7]:
#NOT USED
signal, sample_rate = librosa.load('database\sol\sol_AKZI_FAWCVUT.wav' )
ipd.Audio('database\sol\sol_AKZI_FAWCVUT.wav' )
mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
mfccs_features.shape

(40, 44)

In [8]:
extracted_feataures =[]
for root, directories, files in os.walk(root_path): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            data = mfcc_extractor(os.path.join(root,file_names))
            extracted_feataures.append([data,os.path.basename(os.path.normpath(root))])

    

In [9]:
extracted_feataures_dataframe= pd.DataFrame(extracted_feataures, columns=["mfcc", "word"])
extracted_feataures_dataframe.head()

,mfcc,word
0,"[-466.59473, 54.1608, -32.242916, 11.20391, -1...",ac
1,"[-445.2056, 60.57163, -23.187899, 16.521847, -...",ac
2,"[-419.66544, 68.09416, -13.22027, 10.260144, -...",ac
3,"[-483.0338, 62.193104, -10.424236, 17.89255, -...",ac
4,"[-393.0876, 61.467396, -20.944004, 18.326431, ...",ac


In [10]:
X=np.array(extracted_feataures_dataframe['mfcc'].tolist())
Y=np.array(extracted_feataures_dataframe['word'].tolist())

In [11]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
Y_numerical=to_categorical(labelencoder.fit_transform(Y))

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y_numerical, test_size=0.2, random_state=0)

In [13]:
#NOT USED
X_train.shape

(21187, 40)

In [14]:
#NOT USED
len(Y_test)

5297

ANN Model

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [62]:
model= Sequential()

model.add(Dense(200, input_shape=(40,)))
model.add(Activation('relu'))

model.add(Dense(300))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(Y_numerical.shape[1]))
model.add(Activation('sigmoid'))

In [63]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 200)               8200      
                                                                 
 activation_8 (Activation)   (None, 200)               0         
                                                                 
 dense_9 (Dense)             (None, 300)               60300     
                                                                 
 activation_9 (Activation)   (None, 300)               0         
                                                                 
 dense_10 (Dense)            (None, 50)                15050     
                                                                 
 activation_10 (Activation)  (None, 50)                0         
                                                                 
 dense_11 (Dense)            (None, 14)               

In [64]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [70]:
model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test), verbose=2)

Epoch 1/100
663/663 - 2s - loss: 0.3796 - accuracy: 0.8661 - val_loss: 1.0734 - val_accuracy: 0.7142 - 2s/epoch - 2ms/step
Epoch 2/100
663/663 - 1s - loss: 0.3526 - accuracy: 0.8711 - val_loss: 1.0550 - val_accuracy: 0.7242 - 1s/epoch - 2ms/step
Epoch 3/100
663/663 - 1s - loss: 0.3508 - accuracy: 0.8744 - val_loss: 1.1052 - val_accuracy: 0.7187 - 1s/epoch - 2ms/step
Epoch 4/100
663/663 - 1s - loss: 0.3402 - accuracy: 0.8777 - val_loss: 1.1339 - val_accuracy: 0.7083 - 1s/epoch - 2ms/step
Epoch 5/100
663/663 - 1s - loss: 0.3135 - accuracy: 0.8863 - val_loss: 1.1302 - val_accuracy: 0.7278 - 1s/epoch - 2ms/step
Epoch 6/100
663/663 - 1s - loss: 0.3174 - accuracy: 0.8856 - val_loss: 1.1559 - val_accuracy: 0.7119 - 1s/epoch - 2ms/step
Epoch 7/100
663/663 - 2s - loss: 0.3020 - accuracy: 0.8930 - val_loss: 1.1259 - val_accuracy: 0.7240 - 2s/epoch - 2ms/step
Epoch 8/100
663/663 - 1s - loss: 0.2927 - accuracy: 0.8945 - val_loss: 1.2040 - val_accuracy: 0.7212 - 1s/epoch - 2ms/step
Epoch 9/100
663/

In [71]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 0s 2ms/step - loss: 2.5004 - accuracy: 0.7304
Test Score:  [2.5004451274871826, 0.7304134368896484]
663/663 [==============================] - 1s 2ms/step - loss: 0.0616 - accuracy: 0.9788
Train Score:  [0.06162051856517792, 0.9787605404853821]


PREDICTION WITH DATA

In [72]:
predictiondata_feataures =[]
counter_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/Records"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicted_label=np.argmax(model.predict(rl), axis=-1)
            prediction_class = labelencoder.inverse_transform(predicted_label) 
            predictiondata_feataures.append([os.path.basename(os.path.normpath(root)), prediction_class, predicted_label])
            if os.path.basename(os.path.normpath(root))==prediction_class:
                counter_true+=1

In [73]:
predictiondata_feataures_dataframe= pd.DataFrame(predictiondata_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentage= (counter_true/len(predictiondata_feataures))*100
print("The percantage of the true predicted data: ", percentage,"%")
predictiondata_feataures_dataframe

The percantage of the true predicted data:  45.23809523809524 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[ac],[0]
2,ac,[asagi],[1]
3,asagi,[asagi],[1]
4,asagi,[asagi],[1]
5,asagi,[asagi],[1]
6,baslat,[baslat],[2]
7,baslat,[baslat],[2]
8,baslat,[asagi],[1]
9,devam,[devam],[3]


In [23]:
"""filename="predictionData/dursultan.wav"
r=mfcc_extractor(filename)
print(r)
rl=r.reshape(1,-1)
print(rl)
print(rl.shape)
predicted_label=np.argmax(model.predict(rl), axis=-1)

print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class"""

'filename="predictionData/dursultan.wav"\nr=mfcc_extractor(filename)\nprint(r)\nrl=r.reshape(1,-1)\nprint(rl)\nprint(rl.shape)\npredicted_label=np.argmax(model.predict(rl), axis=-1)\n\nprint(predicted_label)\nprediction_class = labelencoder.inverse_transform(predicted_label) \nprediction_class'